# Downloading PGF UCITS Hedge Share Class NAVs and Holdings

In [1]:
# create a time difference function
import datetime, time

def timediff(start, end, decimals = 1):
    if   int((end - start) / 3600) > 0: # non-zero hours
        return str(int(  (end - start) / 3600)) + 'hr '  + str(int(  (end - start) / 60)          ) + 'min '\
         +                                                 str(round((end - start) % 60, decimals)) + 'sec'
    elif int((end - start) / 60)   > 0: # non-zero hours and minutes
        return str(int(  (end - start) / 60))   + 'min ' + str(round((end - start) % 60, decimals)) + 'sec'
    else:
        return                                             str(round((end - start) % 60, decimals)) + 'sec'

In [2]:
# import libraries

start_time          = time.time()
start_time_overlord = start_time
print('Importing libraries ...')

# load libraries
import pandas as pd
import numpy as np
import os
import csv
import datetime
from datetime import date, datetime
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

print(f'Importing libraries completed: {timediff(start_time, time.time())}')

Importing libraries ...
Importing libraries completed: 0.9sec


In [3]:
# set paths to the driver, urls, and report paramters
start_time = time.time()
print('Setting path strings ...')

os.environ["PATH"] = r'C:/SeleniumDrivers' # + os.pathsep + os.getenv("PATH")
# https://stackoverflow.com/questions/61213005/modify-beginning-of-path-variable-with-os-environ-in-python
eagle_portal       = r'https://eagleportal.prescient.co.za'
url_default        = eagle_portal + r'/Default.aspx'
url_derv           = eagle_portal + r'/Queries/Query.aspx?rpt=DerivativeExposure'
url_parn           = eagle_portal + r'/Queries/Query.aspx?rpt=PortfolioAnalytics'
url_ut_prices      = eagle_portal + r'/Queries/Query.aspx?rpt=UTPRICES'
pth                = r'P:\Investment Operations\GRC\Compliance\Daily\py_reports.xlsm' # variables stored here

print(f'Setting path strings completed: {timediff(start_time, time.time())}')

Setting path strings ...
Setting path strings completed: 0.0sec


In [4]:
# get input data off source sheet

start_time = time.time()
print('Getting input variables ...')

# get source sheet

#get user details, report date, and fund names from the py_report.xlsm sheet
import xlwings as xw
wb      = xw.Book(r'P:\Investment Operations\GRC\Compliance\Daily\py_reports.xlsm') # open calc workbook as an object

# get variables from source sheet and then close the sheet
aladdin        = wb.sheets['creds'].range('A1').value
sesame         = wb.sheets['creds'].range('A2').value
when           = wb.sheets['hdgs'].range('C1').value
when_fmt       = wb.sheets['hdgs'].range('C1').value.strftime('%#m/%#d/%Y') #https://stackabuse.com/how-to-format-dates-in-python/
wb.close()

# get portfolio lists as dataframes, from which to pull reports
df_clss_navs   = pd.read_excel(pth, sheet_name = 'hdgs', usecols = 'A').dropna()  #hedged UT price fund codes
df_clss_hldgs  = pd.read_excel(pth, sheet_name = 'hdgs', usecols = 'B').dropna()  #hedged UT holdings fund codes

# get portfolio lists as strings
clss_navs      = (',').join([element for innerList in df_clss_navs.values  for element in innerList])
clss_hldgs     = (',').join([element for innerList in df_clss_hldgs.values for element in innerList])
# https://stackoverflow.com/questions/12453580/how-to-concatenate-join-items-in-a-list-to-a-single-string

# get elements to manipulate the web calendar
day_           = f'{when:%#d}' # f'{when:%d}' report date with leading zeroes
month_year_    = f'{when:%B}, {when:%Y}'
month_         = f'{when:%b}'
year_          = f'{when:%Y}'

# check inputs
print(f' Report date:          {when}', '\n', f'Class NAV codes:      {clss_navs}', '\n', f'Class holdings codes: {clss_hldgs}')
print(f'Getting input variables completed: {timediff(start_time, time.time())}')

Getting input variables ...
 Report date:          2023-09-22 00:00:00 
 Class NAV codes:      PGPCEMD,PGPCGED,PGPGBFD,PGPGIFE,PGPGIFI,PGPRFE,PGPRFG 
 Class holdings codes: PGPRUSD,PGPRZAR,PGPRUS0%,PGPRGBP,PGIPZAR,PGIP_I,PGPCZAR,PGBEUR,PGBGBP,PGBZAR,PGEMZAR
Getting input variables completed: 3.7sec


In [5]:
# assign the browser driver and log in to the website

start_time = time.time()
print('Assigning webdriver and entering the website ...')

from selenium import webdriver

driver = webdriver.Firefox()
    
# open the browser on the Eagle web page
driver.get(url_default)          # default page
wait = WebDriverWait(driver, 10) # https://selenium-python.readthedocs.io/waits.html, max wait for elements to appear

# login with credentials
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_UserName'   ).send_keys(aladdin)
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_Password'   ).send_keys(sesame)
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_LoginButton').click()
    
print(f'Assigning webdriver and entering the website completed: {timediff(start_time, time.time())}')

Assigning webdriver and entering the website ...
Assigning webdriver and entering the website completed: 19.8sec


In [6]:
# download UT prices

start_time = time.time()
print('Downloading fund NAVs ...')

# having logged in, open the reporting page
driver.get(url_ut_prices)             # ut prices

# switch to the query page
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ModifyLinkLabel'))).click()

edit_criteria_window = driver.window_handles[0] # save current window handle
# How to switch to new window in Selenium for Python?
# https://stackoverflow.com/questions/10629815/how-to-switch-to-new-window-in-selenium-for-python

# update the 'from' calendar
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_B-1"]').click()
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_NMC"]').click()
lmonth_selector = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_PMC"]')))
while driver.find_element(By.XPATH,'//td[@id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_TC"]').text != month_year_:
    lmonth_selector.click()
day_selector = driver.find_element(By.XPATH, f'//td[@class="dxeCalendarDay"][text()={day_}] | //td[@class="dxeCalendarDay dxeCalendarWeekend"][text()={day_}]')
day_selector.click()

# update the 'to' calendar
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_To_B-1"]').click()
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_To_DDD_C_NMC"]').click()
lmonth_selector = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_To_DDD_C_PMC"]')))
while driver.find_element(By.XPATH,'//td[@id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_To_DDD_C_TC"]').text != month_year_:
    lmonth_selector.click()
day_selector = driver.find_element(By.XPATH, f'//td[@class="dxeCalendarDay"][text()={day_}] | //td[@class="dxeCalendarDay dxeCalendarWeekend"][text()={day_}]')
day_selector.click()

# get the web element for the FUND LIST and assign values to it
fund_selector  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedIds"]')
driver.execute_script(f'arguments[0].value = "{clss_navs}";', fund_selector)

# click the table header where "Entity ID" resides
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'table[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedItemsGrid_DXHeaderTable"]'))).click()

time.sleep(20)

# get the web element of the 'Submit' button and then click it
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_RunBtn"]'))).click()

#Wait for and then click the export button and then the xls download button
#https://stackoverflow.com/questions/56085152/selenium-python-error-element-could-not-be-scrolled-into-view
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,  'a[id="DistrBtn"]'        ))).click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ExportMnu_DXI4_T"]'))).click()

print(f'Downloading fund NAVs completed: {timediff(start_time, time.time())}')

In [7]:
# download holdings

start_time = time.time()
print('Downloading hedge class holdings ...')

#having logged in, open the reporting page
driver.get(url_parn)             # fund holdings page

# switch to the query page
driver.find_element(By.CSS_SELECTOR, '#ModifyLinkLabel').click()

edit_criteria_window = driver.window_handles[0] # save curent window handle
# How to switch to new window in Selenium for Python?
# https://stackoverflow.com/questions/10629815/how-to-switch-to-new-window-in-selenium-for-python

# update the calendar
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_B-1"]').click()
driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_NMC"]').click()
lmonth_selector = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_PMC"]')))
while driver.find_element(By.XPATH,'//td[@id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_TC"]').text != month_year_:
    lmonth_selector.click()
day_selector = driver.find_element(By.XPATH,f'//td[@class="dxeCalendarDay"][text()={day_}] | //td[@class="dxeCalendarDay dxeCalendarWeekend"][text()={day_}]')
day_selector.click()

# get the web element for the FUND LIST and assign values to it
fund_selector  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedIds"]')
driver.execute_script(f'arguments[0].value = "{clss_hldgs}";', fund_selector)

# get the web element of the 'Submit' button and then click it
submit_button  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_RunBtn"]')
submit_button.click()

# Wait for and then click the export button and then the xls download button
#https://stackoverflow.com/questions/56085152/selenium-python-error-element-could-not-be-scrolled-into-view
WebDriverWait(driver, 1000).until(EC.element_to_be_clickable((By.CSS_SELECTOR,  'a[id="DistrBtn"]'        ))).click()
WebDriverWait(driver, 1000).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ExportMnu_DXI4_T"]'))).click()

#time.sleep(10) # wait for 10 seconds while the portfolio holdings data xls downloads

print(f'Downloading hedge class holdings completed: {timediff(start_time, time.time())}')
print(f'Roundtrip time to download fund NAVs and hedge class holdings: {timediff(start_time_overlord, time.time())}')

Roundtrip time to download fund NAVs and hedge class holdings: 1min 12.2sec


In [8]:
driver.quit()